In [ ]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.webdriver.common.keys import Keys # para el enter
import os
import re
import regex
import json

from geopy.geocoders import Nominatim


driver = webdriver.Edge()
driver.get('https://scottsdalepublicart.org/public-art/')
driver.maximize_window()


def imagenes():
    time.sleep(2)
    capturas4 = driver.find_elements(By.TAG_NAME, 'h1')  # elementos h
    titulo = capturas4[2].text                           # titulo de la mierda esa
    first_word = titulo.split()[0]
    capturas3 = driver.find_elements(By.TAG_NAME, 'img') # elementos con imagenes 
    titulo2 = 'https://scottsdalepublicart.org/wp-content/themes/scottsdale/library/images/scottsdale-arts-logo.png'
    titulo3 = 'https://scottsdalepublicart.org/wp-content/themes/scottsdale/library/images/orange-pin.png'
    obj2 = []
    
    for n in range(len(capturas3)):  
        s = str(capturas3[n].get_attribute("src")) 
        if s == titulo2:
            k=n+2
            s = str(capturas3[k].get_attribute("src")) 
            while s != titulo3 and (first_word in s or first_word.lower() in s):
                obj2.insert(0,s)
                k= k+1
                s = str(capturas3[k].get_attribute("src"))
                #if k == 6:
                    #break
            break   
    
    return(obj2, titulo)


def parrafo():
    capturas4 = driver.find_elements(By.TAG_NAME, 'h1')  # elementos h
    titulo = capturas4[2].text 
    capturas5 = driver.find_elements(By.TAG_NAME, 'DIV')
    driver.execute_script("window.scrollTo(document.body.scrollHeight,600)")
    driver.find_element(by=By.TAG_NAME, value='body').send_keys(Keys.END)
    rgx = regex.compile(r'(?si)(?|{0}(.*?){1}|{1}(.*?){0})'.format(titulo, '\nMap'))
    s1 = capturas5[55].text
    parrafo=rgx.findall(s1)
    if parrafo == []:
        rgx = regex.compile(r'(?si)(?|{0}(.*?){1}|{1}(.*?){0})'.format(titulo, '\nSHARE\n'))
        s1 = capturas5[55].text
        parrafo=rgx.findall(s1)
    return(parrafo)

def coordenadas():
    capturas15 = driver.find_elements(By.TAG_NAME, 'h3')
    capturas5 = driver.find_elements(By.TAG_NAME, 'DIV')
    rgx = regex.compile(r'(?si)(?|{0}(.*?){1}|{1}(.*?){0})'.format('\nLocation\n', '\nArtist\n'+capturas15[0].text))
    s1 = capturas5[55].text

    location1=rgx.findall(s1)
    rgx = regex.compile(r'(?si)(?|{0}(.*?){1}|{1}(.*?){0})'.format(' [0-9]', ', '))
    ss = str(location1)
    if len(ss)==0: 
        return ('no tiene mapa')

    location2=rgx.findall(ss)
    if len(location2)== 0:
        return ('no tiene mapa') 
    geolocator = Nominatim(user_agent="dsantamaria01@gmail.com")
    location = geolocator.geocode(location2[0])
    return(location.latitude,location.longitude)

def write_json(new_data, filename='page.json'):
    with open(filename,'r+') as file:
      # First we load existing data into a dict.
        file_data = json.load(file)
        # Join new_data with file_data inside emp_details
        file_data["page_details"].append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file, indent = 4)


capturas2 = driver.find_elements(By.TAG_NAME, 'p')

for i in range(14,len(capturas2),2):
    a=[]
    b=[]
    c=[]
    capturas2 = driver.find_elements(By.TAG_NAME, 'p')
    capturas2[i].click()    # todos los objetos que se pueden tener 
    time.sleep(2)
    try:
        (a,titulo) = imagenes()
    except: 
        print('problemas con imagenes en ' + str(i))
    try:
        b = parrafo()
    except: 
        print('problemas con parrafo en ' +str(i))
    try:
        c = coordenadas()
    except: 
        print('problemas con coordenas en ' +str(i))

    # python object to be appended
    y = {"titule": titulo,
         "images": a,
         "paragraph": b,
         "ubication":c
        }

    write_json(y)    
    
    
    print(a, b, c,'estamos en la opcion ' +str(i))        
    time.sleep(2)
    driver.back()
    time.sleep(1)
    
    

